__Основное задание:__
Даны выборки для обучения и для тестирования. Задание заключается в том, чтобы попробовать разные способы валидации, проанализировать плюсы / минусы каждой и сделать выводы о том, какой способ валидации наиболее устойчивый в данной задаче. Метрика качества для оценки прогнозов - ROC-AUC, название целевой переменной - IsFraud. Рекомендуется использовать модели градиетного бустинга, реализация любая / гипепараметры любые. Внимание! выборка assignment_2_test.csv - наш аналог лидерборда. Будем моделировать ситуацию отправки решения на лидерборд и сравнить значение метрики на лидерборде и на локальной валидации. Для других целей использовать выборку запрещено!.
​
Терминалогия, используемая в задании:
* обучающая выборка - выборка, которая передается в метод fit / train;
* валидационная выборка - выборка, которая получается при Hold-Out на 2 выборки (train, valid);
* тестовая выборка - выборка, которая получается при Hold-Out на 3 выборки (train, valid, test);
* ЛБ - лидерборд, выборка assignment_2_test.csv.

In [142]:
from tqdm import tqdm
from typing import List, Tuple

import numpy as np
import pandas as pd
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold, StratifiedKFold, train_test_split, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve, auc
from scipy.stats import probplot

In [2]:
train_data = pd.read_csv(
    "data/assignment_2_train.csv"
)
print("data.shape = {} rows, {} cols".format(*train_data.shape))
train_data.head(n=2)

data.shape = 180000 rows, 394 cols


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
test_data = pd.read_csv(
    "data/assignment_2_test.csv"
)
print("data.shape = {} rows, {} cols".format(*test_data.shape))
test_data.head(n=2)

data.shape = 100001 rows, 394 cols


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3287000,1,7415038,226.0,W,12473,555.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3287001,0,7415054,3072.0,W,15651,417.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
numerical_features = train_data.select_dtypes(include=[np.number])
lb_numerical_features = test_data.select_dtypes(include=[np.number])

In [5]:
target = numerical_features['isFraud']
lb_target = lb_numerical_features['isFraud']

In [6]:
numerical_features = numerical_features.drop(['TransactionID', 'isFraud'], axis=1)
lb_numerical_features = lb_numerical_features.drop(['TransactionID', 'isFraud'], axis=1)

__Задание 1:__ сделать Hold-Out валидацию с разбиением, размер которого будет адекватным, по вашему мнению; разбиение проводить по id-транзакции (TransactionID), обучать модель градиетного бустинга любой реализации с подбором числа деревьев по early_stopping критерию до достижения сходимости. Оценить качество модели на валидационной выборке, оценить расхождение по сравнению с качеством на обучающей выборке и валидационной выборке. Оценить качество на ЛБ, сравнить с качеством на обучении и валидации. Сделать выводы.

In [134]:
X_train, X_valid = train_test_split(numerical_features, train_size=0.75, random_state=42)
y_train, y_valid = train_test_split(target, train_size=0.75, random_state=42)

In [135]:
X_train.shape, X_valid.shape

((135000, 378), (45000, 378))

In [136]:
params = {
    "booster": "gbtree",
    "objective": "binary:logistic",
    "eval_metric": "auc",
    "learning_rate": 0.1,
    "n_estimators": 1000,
    "reg_lambda": 100,
    "max_depth": 4,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

In [137]:
xgb_train = xgb.DMatrix(data=X_train, label=y_train)
xgb_valid = xgb.DMatrix(data=X_valid, label=y_valid)

In [138]:
model1 = xgb.train(
    params=params,
    dtrain=xgb_train,
    num_boost_round=1000,
    early_stopping_rounds=50,
    evals=[(xgb_train, "train"), (xgb_valid, "valid")],
    verbose_eval=50,
    maximize=True,
)

[02:42:20] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.65337	valid-auc:0.63889
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.88480	valid-auc:0.87192
[100]	train-auc:0.90036	valid-auc:0.88795
[150]	train-auc:0.90673	valid-auc:0.89292
Stopping. Best iteration:
[133]	train-auc:0.90669	valid-auc:0.89292



In [68]:
y_train_pred = model1.predict(xgb_train)
y_valid_pred = model1.predict(xgb_valid)

In [69]:
roc_auc_score(y_train, y_train_pred), roc_auc_score(y_valid, y_valid_pred)

(0.9067251551175555, 0.892919719610207)

In [70]:
xgb_lb = xgb.DMatrix(data=lb_numerical_features, label=lb_target)
y_lb_pred = model1.predict(xgb_lb)
roc_auc_score(lb_target, y_lb_pred)

0.8580454525237254

Разница метрики качества на тренировочной и валидационной выборке очень небольшое (1.5%). По такой разнице можно было бы говорить об отсутствии переобучения. Но при переход на ЛБ качество сильно падает, что говорит о том, что модель неустойчива (и мы видели в прошлом д/з, что распределение сумм транзакций на тренировочной и тестовой выборках разное, что в том числе приводит к ухудшению метрики на ЛБ).

__Задание 2:__ сделать Hold-Out валидацию с разбиением на 3 выборки, разбиение проводить по id-транзакции (TransactionID), размер каждой выборки подобрать самостоятельно. Повторить процедуру из п.1. для каждой выборки.

In [45]:
X_train, X_test = train_test_split(numerical_features, train_size=0.8, random_state=42)
X_train, X_valid = train_test_split(X_train, train_size=0.75, random_state=42)
y_train, y_test = train_test_split(target, train_size=0.8, random_state=42)
y_train, y_valid = train_test_split(y_train, train_size=0.75, random_state=42)

In [46]:
xgb_train = xgb.DMatrix(data=X_train, label=y_train)
xgb_valid = xgb.DMatrix(data=X_valid, label=y_valid)
xgb_test = xgb.DMatrix(data=X_test, label=y_test)

In [47]:
model = xgb.train(
    params=params,
    dtrain=xgb_train,
    num_boost_round=1000,
    early_stopping_rounds=50,
    evals=[(xgb_train, "train"), (xgb_valid, "valid"), (xgb_test, "test")],
    verbose_eval=10,
    maximize=True,
)

[02:19:19] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.65507	valid-auc:0.64410	test-auc:0.63977
Multiple eval metrics have been passed: 'test-auc' will be used for early stopping.

Will train until test-auc hasn't improved in 50 rounds.
[10]	train-auc:0.75290	valid-auc:0.73784	test-auc:0.73806
[20]	train-auc:0.82171	valid-auc:0.82093	test-auc:0.81478
[30]	train-auc:0.85931	valid-auc:0.85505	test-auc:0.84872
[40]	train-auc:0.86806	valid-auc:0.86384	test-auc:0.86224
[50]	train-auc:0.88223	valid-auc:0.87520	test-auc:0.87255
[60]	train-auc:0.88862	valid-auc:0.87939	test-auc:0.87758
[70]	train-auc:0.89207	valid-auc:0.88193	test-auc:0.88076
[80]	t

In [48]:
y_train_pred = model.predict(xgb_train)
y_valid_pred = model.predict(xgb_valid)
y_test_pred = model.predict(xgb_test)
roc_auc_score(y_train, y_train_pred), roc_auc_score(y_valid, y_valid_pred), roc_auc_score(y_test, y_test_pred)

(0.9047161422370005, 0.8913999612625666, 0.8912672273354262)

In [49]:
xgb_lb = xgb.DMatrix(data=lb_numerical_features, label=lb_target)
y_lb_pred = model.predict(xgb_lb)
roc_auc_score(lb_target, y_lb_pred)

0.8578127448703862

Качество на тестовой выборке ухудшается по сравнению с валидационной, но разница с качеством на тренировочной выборке все еще небольшая. Качество на ЛБ стало хуже по сравнению с Задачей 1 (возможно из-за того, что модель обучалась на меньшем количестве данных)

__Задание 3:__ построить доверительный интервал на данных из п.2 на основе бутстреп выборок, оценить качество модели на ЛБ относительно полученного доверительного интервала. Сделать выводы.

In [52]:
def create_bootstrap_samples(data: np.array, n_samples: int = 1000) -> np.array:
    """
    Создание бутстреп-выборок.

    Parameters
    ----------
    data: np.array
        Исходная выборка, которая будет использоваться для
        создания бутстреп выборок.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_idx: np.array
        Матрица индексов, для создания бутстреп выборок.

    """
    bootstrap_idx = np.random.randint(
        low=0, high=len(data), size=(n_samples, len(data))
    )
    return bootstrap_idx


def create_bootstrap_metrics(y_true: np.array,
                             y_pred: np.array,
                             metric: callable,
                             n_samlpes: int = 1000) -> List[float]:
    """
    Вычисление бутстреп оценок.

    Parameters
    ----------
    y_true: np.array
        Вектор целевой переменной.

    y_pred: np.array
        Вектор прогнозов.

    metric: callable
        Функция для вычисления метрики.
        Функция должна принимать 2 аргумента: y_true, y_pred.

    n_samples: int, optional, default = 1000
        Количество создаваемых бутстреп выборок.
        Опциональный параметр, по умолчанию, равен 1000.

    Returns
    -------
    bootstrap_metrics: List[float]
        Список со значениями метрики качества на каждой бустреп выборке.

    """
    scores = []

    if isinstance(y_true, pd.Series):
        y_true = y_true.values

    bootstrap_idx = create_bootstrap_samples(y_true)
    for idx in bootstrap_idx:
        y_true_bootstrap = y_true[idx]
        y_pred_bootstrap = y_pred[idx]

        score = metric(y_true_bootstrap, y_pred_bootstrap)
        scores.append(score)

    return scores


def calculate_confidence_interval(scores: list, conf_interval: float = 0.95) -> Tuple[float]:
    """
    Вычисление доверительного интервала.

    Parameters
    ----------
    scores: List[float / int]
        Список с оценками изучаемой величины.

    conf_interval: float, optional, default = 0.95
        Уровень доверия для построения интервала.
        Опциональный параметр, по умолчанию, равен 0.95.

    Returns
    -------
    conf_interval: Tuple[float]
        Кортеж с границами доверительного интервала.

    """
    left_bound = np.percentile(
        scores, ((1 - conf_interval) / 2) * 100
    )
    right_bound = np.percentile(
        scores, (conf_interval + ((1 - conf_interval) / 2)) * 100
    )

    return left_bound, right_bound

In [54]:
np.random.seed(42)
scores = create_bootstrap_metrics(y_test, model.predict(xgb_test), roc_auc_score)

calculate_confidence_interval(scores)

(0.8787561892268496, 0.9036474502323957)

Значение метрики качества на ЛБ не попадает в доверительный интервал, значит модель неустойчива (валидация плохая).

__Задание 4:__ выполнить Adversarial Validation, подобрать объекты из обучающей выборки, которые сильно похожи на объекты из assignment_2_test.csv, и использовать их в качестве валидационного набора. Оценить качество модели на ЛБ, сделать выводы о полученных результатах.

Если столбец TransactionDT оставлять в числе признаков, то модель разделяет выборки абсолютно точно.
Поэтом рассмотрим два варианта: 

1) удалить только столбец TransactionDT

2) оставить только столбец TransactionAmt

In [121]:
X_adv = pd.concat([
    numerical_features, lb_numerical_features], axis=0
)
y_adv = np.hstack((np.zeros(numerical_features.shape[0]), np.ones(lb_numerical_features.shape[0])))
assert X_adv.shape[0] == y_adv.shape[0]

In [122]:
X_adv = X_adv.drop('TransactionDT', axis=1)

In [123]:
model_adv = xgb.XGBClassifier(n_estimators=4)
model_adv.fit(X_adv, y_adv)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=4, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [124]:
y_pred_adv = model_adv.predict_proba(X_adv)
score = roc_auc_score(y_adv, y_pred_adv[:, 1])
print(round(score, 8))

0.85706887


In [125]:
y_pred = model_adv.predict_proba(numerical_features.drop('TransactionDT', axis=1))
y_pred

array([[0.53209764, 0.46790236],
       [0.6158279 , 0.38417208],
       [0.7646603 , 0.23533969],
       ...,
       [0.14235139, 0.8576486 ],
       [0.7742394 , 0.22576055],
       [0.8066354 , 0.19336465]], dtype=float32)

In [126]:
pd.cut(
    y_pred[:, 1], bins=np.arange(0, 1.01, 0.1)
).value_counts().sort_index()

(0.0, 0.1]        0
(0.1, 0.2]    27125
(0.2, 0.3]    93268
(0.3, 0.4]    20554
(0.4, 0.5]    38537
(0.5, 0.6]      135
(0.6, 0.7]       45
(0.7, 0.8]       76
(0.8, 0.9]      260
(0.9, 1.0]        0
dtype: int64

In [127]:
y_pred_lb = model_adv.predict_proba(lb_numerical_features.drop('TransactionDT', axis=1))
y_pred_lb

array([[0.13777602, 0.862224  ],
       [0.6158279 , 0.38417208],
       [0.74375236, 0.25624767],
       ...,
       [0.57889616, 0.42110386],
       [0.73012793, 0.2698721 ],
       [0.14235139, 0.8576486 ]], dtype=float32)

In [128]:
pd.cut(
    y_pred_lb[:, 1], bins=np.arange(0, 1.01, 0.1)
).value_counts().sort_index()

(0.0, 0.1]        0
(0.1, 0.2]     1599
(0.2, 0.3]    13636
(0.3, 0.4]     7867
(0.4, 0.5]    31538
(0.5, 0.6]      842
(0.6, 0.7]     1181
(0.7, 0.8]     1372
(0.8, 0.9]    41957
(0.9, 1.0]        9
dtype: int64

In [129]:
X_train = numerical_features[y_pred[:, 1] < 0.4]
y_train = target[y_pred[:, 1] < 0.4]

X_valid = numerical_features[y_pred[:, 1] >= 0.4]
y_valid = target[y_pred[:, 1] >= 0.4]

In [130]:
xgb_train = xgb.DMatrix(data=X_train, label=y_train)
xgb_valid = xgb.DMatrix(data=X_valid, label=y_valid)

In [131]:
model4_1 = xgb.train(
    params=params,
    dtrain=xgb_train,
    num_boost_round=1000,
    early_stopping_rounds=50,
    evals=[(xgb_train, "train"), (xgb_valid, "valid")],
    verbose_eval=50,
    maximize=True,
)

[02:37:26] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.62234	valid-auc:0.63181
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.88485	valid-auc:0.87499
[100]	train-auc:0.89993	valid-auc:0.88227
[150]	train-auc:0.90490	valid-auc:0.88571
Stopping. Best iteration:
[130]	train-auc:0.90490	valid-auc:0.88576



In [132]:
y_train_pred = model4_1.predict(xgb_train)
y_valid_pred = model4_1.predict(xgb_valid)

In [133]:
roc_auc_score(y_train, y_train_pred), roc_auc_score(y_valid, y_valid_pred)

(0.9049049801930201, 0.8857141353213357)

Вариант 2

In [91]:
X_adv = pd.concat([
    numerical_features, lb_numerical_features], axis=0
)
y_adv = np.hstack((np.zeros(numerical_features.shape[0]), np.ones(lb_numerical_features.shape[0])))
assert X_adv.shape[0] == y_adv.shape[0]

In [96]:
X_adv = X_adv[['TransactionAmt']]

In [97]:
model_adv = xgb.XGBClassifier(n_estimators=4)
model_adv.fit(X_adv, y_adv)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=4, n_jobs=0, num_parallel_tree=1,
              objective='binary:logistic', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [98]:
y_pred_adv = model_adv.predict_proba(X_adv)
score = roc_auc_score(y_adv, y_pred_adv[:, 1])
print(round(score, 8))

0.6331042


In [101]:
y_pred = model_adv.predict_proba(numerical_features[['TransactionAmt']])

array([[0.57853585, 0.42146415],
       [0.5999787 , 0.40002128],
       [0.54255724, 0.4574428 ],
       ...,
       [0.5859054 , 0.41409463],
       [0.54255724, 0.4574428 ],
       [0.56292   , 0.43708   ]], dtype=float32)

In [102]:
pd.cut(
    y_pred[:, 1], bins=np.arange(0, 1.01, 0.1)
).value_counts().sort_index()

(0.0, 0.1]         0
(0.1, 0.2]        86
(0.2, 0.3]     43720
(0.3, 0.4]     23787
(0.4, 0.5]    112379
(0.5, 0.6]        26
(0.6, 0.7]         0
(0.7, 0.8]         2
(0.8, 0.9]         0
(0.9, 1.0]         0
dtype: int64

In [103]:
y_pred_lb = model_adv.predict_proba(lb_numerical_features[['TransactionAmt']])
y_pred_lb

array([[0.56292   , 0.43708   ],
       [0.56827813, 0.43172187],
       [0.5640924 , 0.43590763],
       ...,
       [0.6112087 , 0.38879132],
       [0.5859054 , 0.41409463],
       [0.5640924 , 0.43590763]], dtype=float32)

In [104]:
pd.cut(
    y_pred_lb[:, 1], bins=np.arange(0, 1.01, 0.1)
).value_counts().sort_index()

(0.0, 0.1]        0
(0.1, 0.2]        2
(0.2, 0.3]     5936
(0.3, 0.4]    10806
(0.4, 0.5]    83133
(0.5, 0.6]      103
(0.6, 0.7]        0
(0.7, 0.8]       21
(0.8, 0.9]        0
(0.9, 1.0]        0
dtype: int64

In [105]:
X_train = numerical_features[y_pred[:, 1] < 0.4]
y_train = target[y_pred[:, 1] < 0.4]

X_valid = numerical_features[y_pred[:, 1] >= 0.4]
y_valid = target[y_pred[:, 1] >= 0.4]

In [106]:
xgb_train = xgb.DMatrix(data=X_train, label=y_train)
xgb_valid = xgb.DMatrix(data=X_valid, label=y_valid)

In [107]:
model4_2 = xgb.train(
    params=params,
    dtrain=xgb_train,
    num_boost_round=1000,
    early_stopping_rounds=50,
    evals=[(xgb_train, "train"), (xgb_valid, "valid")],
    verbose_eval=50,
    maximize=True,
)

[02:25:59] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.69303	valid-auc:0.59641
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 50 rounds.
[50]	train-auc:0.92400	valid-auc:0.82114
[100]	train-auc:0.93697	valid-auc:0.84084
[150]	train-auc:0.93747	valid-auc:0.84169
Stopping. Best iteration:
[103]	train-auc:0.93733	valid-auc:0.84191



In [108]:
y_train_pred = model4_2.predict(xgb_train)
y_valid_pred = model4_2.predict(xgb_valid)

In [109]:
roc_auc_score(y_train, y_train_pred), roc_auc_score(y_valid, y_valid_pred)

(0.9374683703264246, 0.8416872576546874)

В первом случае мы видим падение метрики качества на 1.5 пункта, во втором на 9. Это говорит о том, что модель неустройчива и при переходе к ЛБ качество предсказания также может стать ниже на 9 пунктов.

Можно еще model1 из задания 1 применить к данным, "похожим" на ЛБ, но при этом значение метрики меняется не так существенно.

In [139]:
y_valid_pred = model1.predict(xgb_valid)
roc_auc_score(y_valid, y_valid_pred)

0.892919719610207

__Задание 5:__ сделать KFold / StratifiedKFold валидацию (на ваше усмотрение), оценить получаемые качество и разброс по метрике качества. Сделать выводы об устойчивости кросс-валидации, сходимости оценки на кросс-валидации и отложенном наборе данных; Оценить качество на ЛБ, сделать выводы.

In [166]:
def make_cross_validation(X: pd.DataFrame,
                          y: pd.Series,
                          cv_strategy):
  
    models, fold_train_scores, fold_valid_scores = [], [], []
    oof_predictions = np.zeros(X.shape[0])

    for fold_number, (train_idx, valid_idx) in enumerate(cv_strategy.split(X, y)):
        x_train, x_valid = X.loc[train_idx], X.loc[valid_idx]
        y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]

        
        xgb_train = xgb.DMatrix(data=x_train, label=y_train)
        xgb_valid = xgb.DMatrix(data=x_valid, label=y_valid)
        model = xgb.train(
                        params=params,
                        dtrain=xgb_train,
                        num_boost_round=300,
                        early_stopping_rounds=50,
                        evals=[(xgb_train, "train"), (xgb_valid, "valid")],
                        verbose_eval=0,
                        maximize=True,
                    )
        y_train_pred = model.predict(xgb_train)
        y_valid_pred = model.predict(xgb_valid)

        fold_train_scores.append(roc_auc_score(y_train, y_train_pred))
        fold_valid_scores.append(roc_auc_score(y_valid, y_valid_pred))
        oof_predictions[valid_idx] = y_valid_pred

        msg = (
            f"Fold: {fold_number+1}, train-observations = {len(train_idx)}, "
            f"valid-observations = {len(valid_idx)}\n"
            f"train-score = {round(fold_train_scores[fold_number], 4)}, "
            f"valid-score = {round(fold_valid_scores[fold_number], 4)}" 
        )
        print(msg)
        print("="*69)
        models.append(model)

    oof_score = roc_auc_score(y, oof_predictions)
    print(f"CV-results train: {round(np.mean(fold_train_scores), 4)} +/- {round(np.std(fold_train_scores), 3)}")
    print(f"CV-results valid: {round(np.mean(fold_valid_scores), 4)} +/- {round(np.std(fold_valid_scores), 3)}")
    print(f"OOF-score = {round(oof_score, 4)}")

    return models, oof_score, fold_train_scores, fold_valid_scores, oof_predictions

In [167]:
cv_strategy = KFold(n_splits=5, random_state=42)

models, oof_score, fold_train_scores, fold_valid_scores, oof_predictions = make_cross_validation(
    numerical_features, target, cv_strategy=cv_strategy
)

[03:37:35] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Fold: 1, train-observations = 144000, valid-observations = 36000
train-score = 0.9132, valid-score = 0.8706
[03:39:21] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { n_estimators } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Fold: 2, train-observations = 144000, valid-observations = 36000
train-score = 0.9114, valid-score = 0.8918
[03:

Значение метрики при кросс-валидации на валидационных данных ниже, чем при OH разбиении и соответственно ближе к метрики на ЛБ. OOF-score по все выборке крактически совпадает со средним значением метрики на валидационных выборках.

In [ ]:
xgb_lb = xgb.DMatrix(data=lb_numerical_features, label=lb_target)

In [170]:
y_lb_preds = []
for model in models:
    y_lb_pred = model.predict(xgb_lb)
    y_lb_preds.append(y_lb_pred)   

In [172]:
preds = list(zip(*y_lb_preds))

In [179]:
y_lb_pred_avg = []
for pred in preds:
    y_lb_pred_avg.append(np.mean(pred))

In [181]:
roc_auc_score(lb_target, y_lb_pred_avg)

0.861109900963724

Модель, получившаяся усреднением моделей, полученных при кросс-валидации получилась более устройчивая. Значение метрики на ЛБ такой усредненной модели ближе к значениям на валидационных метриках.